## Аггрегация разметки датасета RWSD

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

### Сбор данных разметки и фильтрация разметчиков

Всего в разметку попали 260 объектов, включая, как "старые" данные (до расширения тестового сета), так и "новые" (добавленные в процессе расширения тестового сета).

Виноград прогонялся двумя пулами. В первый пул попали почти все добавленные объекты. Во второй пул попали 4 новых добавленных объектов (но размер второго пула больше из-за контрольных заданий)

In [2]:
assignments1 = pd.read_csv('assignments_from_pool_41009024__19-12-2023.tsv', sep='\t')
assignments2 = pd.read_csv('assignments_from_pool_41266267__19-12-2023.tsv', sep='\t')
skills = pd.read_csv('workerSkills.csv', sep='|')

In [3]:
assignments = pd.concat([assignments1, assignments2[assignments1.columns]])

Разметчикам предлагалось на основании имеющихся текст, фраз со словом и с предлогом из данного текста ответить на вопрос, относятся ли обе фразы к одной и той же сущности.

Вход: 
- INPUT:text (пример: `Пршел к Владу и Маше в гости. Она открыла дверь`).
- INPUT:word (пример: `Маша`).
- INPUT:pronoun (пример: `Она`).

Выход:
- OUTPUT:result (булеан: `True` или `False`).

In [4]:
assignments.head(1)

,INPUT:text,INPUT:word,INPUT:pronoun,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,Боб заплатил за обучение Чарльза в университет...,Боб,Он,True,True,NaN,NaN,https://platform.toloka.ai/task/41009024/00027...,000271bf80--64ef36132409351a10c5f66d,000271bf80--64ef38197c475157010635d8,000271bf80--64ef38187c475157010635d2,36d93d4f906f48015a04300bb61d8ff1,APPROVED,2023-08-30T12:37:45,2023-08-30T12:38:29.678,2023-08-30T12:38:29.678,0.019


Фильтруем толокеров, которые дали меньше половины корректных ответов на контрольных заданиях.

In [5]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    text = row[0]
    word = row[1]
    pron = row[2]

    out = row[3]
    
    gold = row[4]

    user = row[11]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  155
Bad users: 5


5 из 155 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Теперь нужно оставить только основной пул. Контрольные задания брались из трейна Винограда, потому их нужно убрать. На контрольных заданиях есть `GOLDEN:result`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:text` содержит NaN.

In [6]:
assignments_no_control = assignments[assignments['GOLDEN:result'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:text'].notnull()]

Посчитаем, сколько было затрачено на получение разметки тестовых данных без учета контрольных заданий, так как они могли проходиться неограниченное количество раз одним и тем же разметчиком.

In [7]:
def w_sum(df):
    idx = df.index.values
    vals = df.values
    summ = idx * vals
    return summ.sum()

d1 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts(normalize=True)
d2 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts()
print(f'взвешенная цена айтема в тесте: {round(w_sum(d1), 3)}')
print(f'потрачено на разметку теста: {round(w_sum(d2), 3)}')
print(f'{round(w_sum(d2), 3)} / {round(w_sum(d1), 3)}')

взвешенная цена айтема в тесте: 0.021
потрачено на разметку теста: 27.046
27.046 / 0.021


Выделим, сколько составила средняя часовая ставка для разметки тестовой части датасета. Это будет простое среднее из следующих величин: количество заданий, которое разметчк может сделать за час на основе данного задания, помноженное на цену задания.

In [8]:
def get_hour_pay(df):
    try:
        times = pd.to_datetime(df['ASSIGNMENT:submitted']) - pd.to_datetime(df['ASSIGNMENT:started'])
    except Exception as e:
        times = []
        for i in range(len(df)):
            try:
                start = pd.to_datetime(df['ASSIGNMENT:started'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]).iloc[i])
            try:
                end = pd.to_datetime(df['ASSIGNMENT:submitted'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1]).iloc[i])
            delta = end - start
            times.extend([delta])
        times = pd.Series(times)
        # times = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1])) - pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]))
    sums = 3600 / times.apply(lambda x: x.seconds) * df['ASSIGNMENT:reward']
    return sums.mean()

get_hour_pay(assignments_no_control_no_null)

1.4836675506254484

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [9]:
from collections import defaultdict

text_dict = defaultdict(list)

for text, word, pron, user, out in zip(
    assignments_no_control_no_null["INPUT:text"], assignments_no_control_no_null["INPUT:word"],
    assignments_no_control_no_null["INPUT:pronoun"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:result"]
    ):
    if user not in bad_users:
        text_dict[(text, word, pron)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

260


In [10]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 250, 4: 10})

Есть 10 заданий с перектрытием 4. Для формирования итоговых лейблов нужно, чтобы было простое большинство разметчиков, проголосовавших за данную опцию. Если большинства нет, то оценка строится, исходя из оценки навыков разметчиков. В таком случае, финальный лейбл будет присвоен по голосу группы с наилучшими навыками.

In [14]:
preds_full = {}
user2skill = {k:v for k, v in zip(skills['worker_id'], skills['skill_value'])}
control_acc = assignments[assignments['GOLDEN:result'].notna()]\
    .groupby('ASSIGNMENT:worker_id')\
        .apply(lambda x: (np.array(x['OUTPUT:result']) == np.array(x['GOLDEN:result'])).mean())
user2control = {k:v for k, v in zip(control_acc.index, control_acc.values)}

from crowdkit.aggregation.classification.glad import GLAD

full = assignments['INPUT:text'] + ' ' + assignments['INPUT:word'] + ' ' + assignments['INPUT:pronoun']
id2task = dict(enumerate(full))
task2id = {k:v for v, k in id2task.items()}
id2user = dict(enumerate(assignments['ASSIGNMENT:worker_id']))
user2id = {k:v for v, k in id2user.items()}

codes = full.map(task2id)
res = pd.DataFrame({'task': codes, 'worker': assignments['ASSIGNMENT:worker_id'].map(user2id), 'label': assignments['OUTPUT:result'].astype(int)})
model = GLAD(n_iter=10000, tol=1e-06, m_step_max_iter=1000, m_step_tol=1e-03)
model.fit(res)
user2alpha = dict(enumerate(model.alphas_))
tb = model.alphas_.copy()
tb.index = tb.index.map(id2user)
user2alpha = {k:v for k, v in zip(tb.index, tb.values)}

stats = {
    'total_agreement': 0,
    'majority': 0,
    'skill_based': 0,
    'major_based': 0,
    'em_based': 0,
    'rest': 0,
}

for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [[ans[j][0], ans[j][1]['out']] for j in range(len(ans))]
    users, votes = list(zip(*lst))
    cnt = pd.Series(Counter(votes)).sort_values(ascending=False)

    # # total agreement
    if len(cnt) == 1:
        res = cnt.index[0]
        stats['total_agreement'] += 1
    # simple majority
    elif cnt.iloc[0] > cnt.iloc[1]:
        res = cnt.index[0]
        stats['majority'] += 1
    # (> 1 options) & (1 option == 2 option)
    else:
        # try overall skill based comparison
        vals = list(map(lambda x: user2skill[x], users))
        table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
        agg = table.groupby('votes').agg(
            sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
            sum_votes=pd.NamedAgg(column='user', aggfunc='count')
        ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
        # check there is a leader by skills
        if agg['sum_skill'].iloc[0] > agg['sum_skill'].iloc[1]:
            res = agg.index[0]
            stats['skill_based'] += 1
        else:
            # top-3 answers by overall skills
            vals = list(map(lambda x: user2skill[x], users))
            table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
            table = table.sort_values(by='skill', ascending=False)
            if len(table) >= 3:
                sub = table.iloc[:3]
            else:
                sub = table
            agg = sub.groupby('votes').agg(
                sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                sum_votes=pd.NamedAgg(column='user', aggfunc='count')
            ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
            if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                res = agg.index[0]
                stats['major_based'] += 1
            
            else:
                vals = list(map(lambda x: user2alpha[x], users))
                table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
                agg = table.groupby('votes').agg(
                    sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                    sum_votes=pd.NamedAgg(column='user', aggfunc='count')
                ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
                # check there is a leader by skills
                if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                    res = agg.index[0]
                    stats['em_based'] += 1
                else:
                    res = agg.index[0]
                    stats['rest'] += 1

    preds_full[keys[i]] = res

In [15]:
stats

{'total_agreement': 137,
 'majority': 122,
 'skill_based': 1,
 'major_based': 0,
 'em_based': 0,
 'rest': 0}

Только в 1 объекте согласованность не была достигнута. Убираем его.

In [16]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['text', 'span1', 'span2']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными метками.

In [17]:
res_df = pd.read_csv('winograd.csv')

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [18]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['text'] = res_df['text'].apply(format_text)
res_df['span1'] = res_df['span1'].apply(format_text)
res_df['span2'] = res_df['span2'].apply(format_text)

preds_full_df['text'] = preds_full_df['text'].apply(format_text)
preds_full_df['span1'] = preds_full_df['span1'].apply(format_text)
preds_full_df['span2'] = preds_full_df['span2'].apply(format_text)

res_df['full'] = res_df['text'] + ' ' + res_df['span1'] + ' ' + res_df['span2']
preds_full_df['full'] = preds_full_df['text'] + ' ' + preds_full_df['span1'] + ' ' + preds_full_df['span2']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [19]:
new = res_df.merge(preds_full_df.drop(['text', 'span1', 'span2'], axis=1), on='full', how='left')

In [20]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

260

In [21]:
new_valid.head(5)

,text,span1,span2,lb_x,full,lb_y
0,"Мод и Дора видели, как через прерию несутся по...",Мод и Дора,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
1,"Мод и Дора видели, как через прерию несутся по...",поезда,они появились,True,"Мод и Дора видели, как через прерию несутся по...",True
2,"Мод и Дора видели, как через прерию несутся по...",клубы,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
3,"Мод и Дора видели, как через прерию несутся по...",Ревущие звуки,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
4,"Мод и Дора видели, как через прерию несутся по...",свистки,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False


### Подсчет метрики

Если в правом столбце меток осталось 259 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось.

In [22]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [23]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.8346153846153846

`Accuracy = 0.835`